# Training NLU Model

In [1]:
from rasa_nlu.training_data import load_data
from rasa_nlu import config
from rasa_nlu.model import Trainer, Metadata, Interpreter

In [2]:
def train_nlu(data,configuration,model_dir):
    training_data = load_data(data)
    trainer = Trainer(config.load(configuration))
    trainer.train(training_data)
    model_directory = trainer.persist(model_dir,fixed_model_name='weathernlu')

In [3]:
train_nlu_model_switch = False

In [4]:
if (train_nlu_model_switch):
    train_nlu('./data/data.json','config_spacy.json','./models/nlu')

In [5]:
def run_nlu(query):
    interpreter = Interpreter.load('./models/nlu/default/weathernlu')
    return interpreter.parse(query)

In [6]:
query = "I am planning a holiday to India. I wonder how is the weather out there"

In [7]:
parsed_query = run_nlu(query)
query_intent = parsed_query['intent']['name']
query_entity = parsed_query['entities'][0]['value']
print('Intent: {} \n Entity: {}'.format(query_intent,query_entity))

C:\Users\Utkarsh Sah\Anaconda3\lib\site-packages\rasa_nlu\extractors\entity_synonyms.py:85: UserWarning: Failed to load synonyms file from './models/nlu/default/weathernlu\entity_synonyms.json'
  "".format(entity_synonyms_file))


Intent: inform 
 Entity: india


In [8]:
parsed_query

{'entities': [{'confidence': 0.7114679175622723,
   'end': 32,
   'entity': 'location',
   'extractor': 'ner_crf',
   'start': 27,
   'value': 'india'}],
 'intent': {'confidence': 0.80780585954411999, 'name': 'inform'},
 'intent_ranking': [{'confidence': 0.80780585954411999, 'name': 'inform'},
  {'confidence': 0.12270394872887903, 'name': 'goodbye'},
  {'confidence': 0.069490191727000813, 'name': 'greet'}],
 'text': 'I am planning a holiday to India. I wonder how is the weather out there'}

# Custom Action

In [9]:
from __future__ import absolute_import
from __future__ import division
from __future__ import unicode_literals

from rasa_core.actions.action import Action
from rasa_core.events import SlotSet

In [18]:
class ActionWeather(Action):
    def name(self):
        return 'action_weather'
    
    def run(self, dispatcher, tracker, domain):
        from apixu.client import ApixuClient
        api_key = '5ed103a862bd442182385105181306'
        client = ApixuClient(api_key)
        loc = tracker.get_slot('location')
        current = client.getCurrentWeather(q=loc)
        city = current['location']['name']
        condition = current['current']['condition']['text']
        response = """It is currently {} in {} at the moment.""".format(condition,city)
        dispatcher.utter_message(response)
        return [SlotSet('location',loc)]

# Training Dialog Management System

In [20]:
import logging

from rasa_core.agent import Agent
from rasa_core.policies import KerasPolicy
from rasa_core.policies.memoization import MemoizationPolicy

In [29]:
train_dialog_management_switch = False

In [28]:
if (train_dialog_management_switch):
    logging.basicConfig(level='INFO')
    training_data_file = './data/stories.md'
    model_path = './models/dialogue'
    agent = Agent('weather_domain.yml', policies=[MemoizationPolicy(), KerasPolicy()])
    agent.train(training_data_file,
                augmentation_factor=50,
                #max_history=2,
                epochs=500,
                batch_size=10,
                validation_split=0.2)
    agent.persist(model_path)

Processed Story Blocks: 100%|████| 4/4 [00:00<00:00, 125.00it/s, # trackers=20]
INFO:rasa_core.featurizers:Creating states and action examples from collected trackers (<class 'rasa_core.featurizers.MaxHistoryTrackerFeaturizer'>)...
Processed trackers: 100%|█████| 152/152 [00:02<00:00, 54.99it/s, # actions=169]
INFO:rasa_core.featurizers:Created 169 action examples.
Processed actions: 169it [00:00, 394.83it/s, # examples=127]
INFO:rasa_core.policies.memoization:Memorized 127 unique action examples.
INFO:rasa_core.featurizers:Creating states and action examples from collected trackers (<class 'rasa_core.featurizers.MaxHistoryTrackerFeaturizer'>)...
Processed trackers: 100%|█████| 152/152 [00:02<00:00, 53.52it/s, # actions=169]
INFO:rasa_core.featurizers:Created 169 action examples.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_1 (Masking)          (None, 5, 11)             0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                5632      
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 198       
_________________________________________________________________
activation_1 (Activation)    (None, 6)                 0         
Total params: 5,830
Trainable params: 5,830
Non-trainable params: 0
_________________________________________________________________


INFO:rasa_core.policies.keras_policy:Fitting model with 169 total samples and a validation split of 0.2


Train on 135 samples, validate on 34 samples
Epoch 1/500
135/135 [==============================] - ETA: 11s - loss: 1.7955 - acc: 0.1000 - ETA: 0s - loss: 1.7496 - acc: 0.3300  - 1s 8ms/step - loss: 1.7312 - acc: 0.4074 - val_loss: 1.6966 - val_acc: 0.3824
Epoch 2/500
135/135 [==============================] - ETA: 0s - loss: 1.6506 - acc: 0.4000 - ETA: 0s - loss: 1.6198 - acc: 0.5538 - 0s 563us/step - loss: 1.6160 - acc: 0.5556 - val_loss: 1.6393 - val_acc: 0.3824
Epoch 3/500
135/135 [==============================] - ETA: 0s - loss: 1.5649 - acc: 0.6000 - ETA: 0s - loss: 1.5239 - acc: 0.5462

INFO:rasa_core.policies.keras_policy:Done fitting keras policy model
INFO:rasa_core.agent:Persisted model to 'C:\Users\Utkarsh Sah\Desktop\rasa_nlu\models\dialogue'
